#This project encompasses the LimeLocale's Finance API beta official AI that assists the team in Research in Stock health per company all over the world

#As a starting step, It currently automates the Team's research on Stock and Crypto in China's GBA
#TODO-Measure the accuracy of the model


In [1]:
#Baseline_Setup_and_dependencies
#Pegasus Tokenizer, NLP for summarization
#BS4 scraps the web for news
import sentencepiece
import tensorflow
import requests
import re
import tensorflow
from transformers import PegasusTokenizer,PegasusForConditionalGeneration, TFPegasusForConditionalGeneration
from bs4 import BeautifulSoup
from transformers import pipeline


2023-04-07 18:56:33.007674: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-07 18:56:33.044656: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-07 18:56:33.045516: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-07 18:56:33.796379: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/misango/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Loading and saving the summarization models  from the Hugging face model

model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

'''
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("human-centered-summarization/financial-summarization-pegasus")
model = AutoModelForSeq2SeqLM.from_pretrained("human-centered-summarization/financial-summarization-pegasus")
'''

'\nfrom transformers import AutoTokenizer, AutoModelForSeq2SeqLM\ntokenizer = AutoTokenizer.from_pretrained("human-centered-summarization/financial-summarization-pegasus")\nmodel = AutoModelForSeq2SeqLM.from_pretrained("human-centered-summarization/financial-summarization-pegasus")\n'

This Code slices focus on the Automation of more than one stock symbol

In [4]:
#Building a News and Sentiment Pipleline
#Test with company stock names --Edited to user preference
#The model asks which tinkers you wanna monitor and how much
#You can also edit the search URL to use expedia or Bloomberg finance
#Search for stock news using Google and Yahoo Finance
#TODO--> Figure out how to allow a universal Web scrapping instead of relying solely on Yahoo finance. ie investopedia, Bloomberg LP etc
def search_for_stock_news_urls(ticker):
  search_url = f"https://www.google.com/search?q=yahoo+finance+{ticker}&tbm=nws"
  r = requests.get(search_url)
  soup = BeautifulSoup(r.text, 'html.parser')
  atags = soup.find_all('a')
  hrefs = [link['href'] for link in atags]
  return hrefs

num_tickers = int(input("Enter the number of stock tickers you want to monitor: "))
monitored_tickers = []
for i in range(num_tickers):
  ticker = input(f"Enter the company ticker {i+1}: ")
  monitored_tickers.append(ticker)

raw_urls = {ticker: search_for_stock_news_urls(ticker) for ticker in monitored_tickers}
raw_urls

{'TSLA': ['/?sa=X&ved=0ahUKEwjFjYTHz5f-AhVZNsAKHW4lCDYQOwgC',
  '/search?q=yahoo+finance+TSLA&tbm=nws&ie=UTF-8&gbv=1&sei=dPgvZMWDJNnsgAbuyqCwAw',
  '/search?q=yahoo+finance+TSLA&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwjFjYTHz5f-AhVZNsAKHW4lCDYQ_AUIBSgA',
  '/search?q=yahoo+finance+TSLA&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwjFjYTHz5f-AhVZNsAKHW4lCDYQ_AUIBygC',
  'https://maps.google.com/maps?q=yahoo+finance+TSLA&um=1&ie=UTF-8&sa=X&ved=0ahUKEwjFjYTHz5f-AhVZNsAKHW4lCDYQ_AUICCgD',
  '/search?q=yahoo+finance+TSLA&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwjFjYTHz5f-AhVZNsAKHW4lCDYQ_AUICSgE',
  '/search?q=yahoo+finance+TSLA&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwjFjYTHz5f-AhVZNsAKHW4lCDYQ_AUICigF',
  '/search?q=yahoo+finance+TSLA&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwjFjYTHz5f-AhVZNsAKHW4lCDYQ_AUICygG',
  '/advanced_search',
  '/search?q=yahoo+finance+TSLA&ie=UTF-8&tbm=nws&source=lnt&tbs=lr:lang_1zh-CN%7Clang_1zh-TW&lr=lang_zh-CN%7Clang_zh-TW&sa=X&ved=0ahUKEwjFjYTHz5f-Ah

In [5]:
#Strip out the unwanted URLs--> policies,accounts, preferences etc
excluded_list = ['maps', 'policies', 'preferences', 'support', 'accounts']
def strip_unwanted_urls(urls, excluded_list):
    final_val = []
    for url in urls:
        if 'https://' in url and not any(excluded_word in url for excluded_word in excluded_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            final_val.append(res)
    return list(set(final_val))

cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], excluded_list) for ticker in monitored_tickers}
cleaned_urls


{'TSLA': ['https://finance.yahoo.com/news/forget-tesla-biggest-tech-opportunities-220000980.html',
  'https://au.finance.yahoo.com/news/tesla-china-boss-tom-zhu-034428650.html',
  'https://finance.yahoo.com/news/special-report-tesla-workers-shared-135329460.html',
  'https://uk.finance.yahoo.com/news/3-reasons-m-watching-tesla-110011915.html',
  'https://finance.yahoo.com/news/everybodys-getting-into-the-ev-game-edmunds-185003513.html',
  'https://finance.yahoo.com/news/ford-ceo-tesla-is-going-to-see-more-price-competition-in-the-ev-market-154750743.html',
  'https://finance.yahoo.com/news/tesla-lays-five-proposals-annual-205736683.html',
  'https://finance.yahoo.com/news/tesla-iron-based-batteries-semi-200347655.html',
  'https://finance.yahoo.com/news/1-tesla-cuts-prices-us-051344969.html',
  'https://www.google.com/search?q%3Dyahoo%2Bfinance%2BTSLA%26tbm%3Dnws%26pccc%3D1',
  'https://finance.yahoo.com/news/ev-roundup-tsla-q1-deliveries-141002509.html']}

In [6]:
#Seach and scrape cleaned URLs
def scrape_and_process(urls):
    ARTICLES = []
    for url in urls:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = [paragraph.text for paragraph in paragraphs]
        words = ''.join(text).split(' ')[:400]
        ARTICLE = ''.join(words)
        ARTICLES.append(ARTICLE)
    return ARTICLES

articles = {ticker:scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_tickers}
articles

{'TSLA': ['Thetechmarkethasyettobottomout,andinvestorsshouldbeavoidingthemega-capsatallcosts,butthereareemergingstarsfromAIandgamingtosemiconductorsthatunderpinthenewtechtrends–andthey’reundervalued,notovervalued.Tesla(NASDAQ:TSLA)istanking.Inthepast12months,thestockhasshednearly30%,withElonMusktryingtoexplainawaythenosediveformonths.Thetechmarketisaseaofredrightnow,andit’sdraggingWallStreetdownwithit.Shopify(NYSE:SHOP)haslostnearly42%inthepastyear.Coinbase(NASDAQ:COIN)hastankednearly70%inthesametimeperiod.EvenMeta(NASDAQ:META)hasshedalmost20%.Thisisnowatimeofreflection,whereinvestorsshouldbelookingdeeperintothelessobviouscornersofthetechworld,wherebrilliantthingsarebeingaccomplished,andwherevaluationsarestillverylow.Thereisonlydownsideinthemega-captechstocks,butweseeallkindsofupsideinsomelesser-knownstocksintheAI,gaming,andsemiconductorplayingfields.#1ChatbotGPTtrailblazerBigBear.ai(NYSE:BBAI)ChatbotGPTisthehottestAIbuzzwordrightnow,andanysoftwarecompanyassociatedwithithasbeensurfingt

In [7]:
from transformers import pipeline

# Define the summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def summarize_all_articles(articles):
    summaries = []
    for article in articles:
        # Use the summarizer pipeline to generate a summary
        summary = summarizer(article, max_length=120, min_length=30, do_sample=False)[0]['summary_text']
        summaries.append(summary)
    return summaries

summaries = {ticker:summarize_all_articles(articles[ticker]) for ticker in monitored_tickers}


Your max_length is set to 120, but you input_length is only 27. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)


In [8]:
#Calling the sentiment analysis to perform sentiment labelling for the summaries
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model_revision = "af0f99b"
model = AutoModelForSequenceClassification.from_pretrained(model_name, revision=model_revision)
tokenizer = AutoTokenizer.from_pretrained(model_name, revision=model_revision)
sentiment = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

In [15]:
#Adding Sentiment Analysis to give a Negative/Positive score
#-VE means the market is not good for the product
#+VE means the market is good for the product/investment
scores = {
    ticker: sentiment(summaries[ticker])
    for ticker in monitored_tickers
}
print(summaries)
print(scores)
#The model counts the -VE sentiment in each score dictionary
#If the scores are more than 5, the model predicts you dont buy the stock
#The model only counts the NEGATIVE sentiments
for ticker in scores:
    negative_count = sum(
        1 for score in scores[ticker] if score['label'] == 'NEGATIVE'
    )
    if negative_count > 5:
      print(f"The model finds that stock {ticker} is not doing well currently, We recommend that you dont buy for short holding")
    elif negative_count == 5:
      print(f"The model generates a neutral view on stock {ticker}, further research is needed : Bloomberg / Expedia refinement")
    elif negative_count < 5:
      print(f"The model finds that stock {ticker} is Good to buy for the short term, contact our brokers to buy ")


{'TSLA': ['Thetech market isaseaofredrightnow, and it’sdraggingWallStreetdownwithit. Thetechmarkethasyettobottomout, andinvestorsshouldbeavoidingthemega-capsatallcosts. But there areemergingstarsfromAIandgamingtosemiconductorsthatunderpinthenewtechtrends.', "China delivered88,869vehicles in March. Thatmeansmorethanhalfofthecarmaker'sQ1deliveriesoriginatedfromitsShanghaifactory.", 'GroupsofTeslaemployeesprivatelysharedvia aninternalmessagingsystem. Sometimeshighlyinvasivevideosandimagesrecordedbycustomers’carcameras.', 'Tesla(NASDAQ:TSLA)stock has beenafavouriteofinvestors foranumberofyears. Manyfear thatTeslastockisoverhyped,leadingtoanexcessiveprice-to-earnings(P/E)ratio of52. However,Iprefertofocusontheperformanceofthecompanyitself.Ihavethreekeyreasons why I’mexcitedforwhatTeslahasinstore.', "AlistairWeaver,editor-in-chiefofEdmunds,saysthosedaysareover. “There'salot moreproductcomingintothemarket,” weaverrecentlytoldYahooFinance.", "Ford(F) CEO JimFarley told Tesla(TSLA)investors: Yo

This slice is for exporting a summary

In [18]:
#Step-1 is to summarize the blueprint labels for the CSV
#show why the previous tinker is not good to buy
def create_output_array(summaries, scores, urls):
  output = []
  for ticker in monitored_tickers:
    for counter in range(len(summaries[ticker])):
      output_this = [
      ticker,
      summaries[ticker][counter],
      scores[ticker][counter],
      urls[ticker][counter],
    ]
    output.append(output_this)
  return output

final_output = create_output_array(summaries,scores,cleaned_urls)
final_output

[['TSLA',
  'NikolaCorpNKLAalsomadeheadlines asitcelebrated100saleordersforClass8FCEV. However,sharesofthecompanywerebrutallypunished afterthecompanyannounceda$100millionstocksaleplan. TeslacarriesaZacksRank#4(Sell)and NIOis#5 ranked (StrongSell).',
  {'label': 'NEGATIVE', 'score': 0.9904288053512573},
  'https://finance.yahoo.com/news/ev-roundup-tsla-q1-deliveries-141002509.html']]

In [19]:
#Exporting the results
#TOCO--> Fix the csv output
final_output.insert(0, ['Ticker', 'Summary', 'Label', 'Confidence', 'Url'])
import csv
with open('assetsummaries.csv', mode='w', newline='') as f:
  csv_writer = csv.writer(f, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
  csv_writer.writerows(final_output)